In [9]:
import urllib.request
import pandas as pd
import numpy as np
from nltk import word_tokenize

# load the Quora questions dataset and preprocess to clean the data
df = pd.read_csv("quora_duplicate_questions.tsv",delimiter='\t')
df = df.drop(['id','qid1','qid2'],axis=1) # don't need the ID columns (ID also provided by DF object)

to_drop = [] # list to drop any NaN or non-tokenizable rows found during scan of all tokens

for i,row in df.iterrows():
    try:
        tok_text = word_tokenize(row['question1']) + word_tokenize(row['question2'])
    except: # drop all rows which can't be tokenized by nltk, e.g. NaN
        to_drop.append(i)
        
for i in to_drop:
    df = df.drop([i])

print(df.shape)
print("preprocessing complete")

(404290, 3)
preprocessing complete


In [2]:
'''
search for named entity overlap and add this ratio to df.  This can help identify content-similar questions, but 
will be tricked by non-duplicates with differing question words, e.g. "how do I go from New York to Kentucky?" vs.
"Why would I go from New York to Kentucky?"
'''
from nltk import pos_tag, ne_chunk
from nltk.chunk import tree2conlltags

# add new column for entity overlap feature
def add_entity_col(df):
    df['entity overlap'] = np.zeros(len(df))
    return df

# takes the input texts and returns chunks
def get_chunks(text):
    return tree2conlltags(ne_chunk(pos_tag(word_tokenize(text))))

# returns named entity chunks from all input chunks
def get_named_entities(chunk):
    entities = []
    for token in chunk:
        if token[2] != 'O':
            entities.append(token[0])
    return entities

# takes list of entities and returns entity overlap / total number of entities
def get_entity_overlap(list_one,list_two):
    num_entities = len(list_one+list_two)
    count = 0
    for entity in list_one:
        if entity in list_two: # same entity in both NE lists
            count +=1
            
    try: 
        ratio = count/num_entities
    except: # catch Divide by 0 error in case there were no entities
        ratio = 0
        
    return ratio # ratio of same entities to total number of entities in both lists

# add the named entity column to the df and fill in its value
def fill_named_entity(df):
    df = add_entity_col(df)
    
    for i,row in df.iterrows():
        chunk_one = get_chunks(row['question1'])
        list_one = get_named_entities(chunk_one)
    
        chunk_two = get_chunks(row['question2'])
        list_two = get_named_entities(chunk_two)
    
        entity_overlap = get_entity_overlap(list_one,list_two)
    
        df.loc[i,'entity overlap'] = entity_overlap
    
    return df

In [3]:
'''
search for question words overlap and add this radio to df.  This can help to combat the problem listed above, i.e.
pairs which use the same question words are more likely to be going after the same answer.
question words = what/which, when, where, who/whom, why, how
'''
# adds a new column for each question word in the df
def add_question_cols(df):
    question_words = ['what/which','when','where','who/whom','why','how']

    for word in question_words:            
        df[word] = np.zeros(len(df))
    return df, question_words

# fill in the question word columns for each row with occurrences of question word / total number of words
def fill_question_words(df):
    df, question_words = add_question_cols(df)
    
    for i,row in df.iterrows():
        one_tok = word_tokenize(row['question1'])
        one_tok = [word.lower() for word in one_tok]
        two_tok = word_tokenize(row['question2'])
        two_tok = [word.lower() for word in two_tok]
        
        num_words = len(one_tok+two_tok)
        
        for question_word in question_words:
            occurrences = 0
            if '/' in question_word: # check for multiple words under one category, e.g. who/whom, and search for both
                word_list = question_word.split('/')
                for word in word_list:
                    occurrences += one_tok.count(word) + two_tok.count(word)
            else:
                occurrences = one_tok.count(question_word) + two_tok.count(question_word)
            df.loc[i,question_word] = occurrences/num_words
        
    return df
    

In [4]:
'''
search for n-gram similarity features and add them to the df.  These features will also pick up on named entities
and question words like above.  Jaccard distance and cosine similarity added to df.
'''
from nltk.util import ngrams
from collections import Counter
import math
import numpy as np

# adds 2 columns, Jaccard and cosine, for each n-gram value in range 1-6
def add_ngram_cols(df):
    n_gram = list(range(1,7))
    
    for gram in n_gram:
        df[str(gram)+"-gram cosine"] = np.zeros(len(df))
        df[str(gram)+"-gram jaccard"] = np.zeros(len(df))
    return df, n_gram
    
def get_ngrams(text, gram):
    grams = list(ngrams(text.split(), gram,pad_left=True,pad_right=True))
    return grams

def cosine_similarity_ngrams(a, b):
    vec1 = Counter(a)
    vec2 = Counter(b)
    
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    return float(numerator) / denominator

def jaccard_distance_ngrams(a, b):
    a = set(a)
    b = set(b)
    return 1.0 * len(a&b)/len(a|b)

# fill in the Jaccard distance and cosine similarity for each ngram 1-6
def fill_ngrams(df):
    df, n_gram = add_ngram_cols(df)
    
    for i,row in df.iterrows():
        for gram in n_gram:
            one_ngrams = get_ngrams(row['question1'],gram)
            two_ngrams = get_ngrams(row['question2'],gram)
            cosine_similarity = cosine_similarity_ngrams(one_ngrams,two_ngrams)
            jaccard_distance = jaccard_distance_ngrams(one_ngrams,two_ngrams)
        
            df.loc[i,str(gram)+"-gram cosine"] = cosine_similarity
            df.loc[i,str(gram)+"-gram jaccard"] = jaccard_distance
            
    return df

In [5]:
'''
iterate through df and fill columns.
NOTE: runtime of about 1 day to generate complete dataset with all features added
'''
import datetime as dt

def fill_dataframe(df,filename):
    print("starting ",filename)
    
    start = dt.datetime.now()
    
    df = fill_named_entity(df)
    print("named entities complete for ",filename,", took ",dt.datetime.now()-start, " seconds")
    print(df.head())
    
    df = fill_question_words(df)
    print("question words complete for ",filename,", took ",dt.datetime.now()-start, " seconds")
    print(df.head())
    df = fill_ngrams(df)
    print("n-grams complete for ",filename,", took ",dt.datetime.now()-start, " seconds")
    print(df.head())
    end = dt.datetime.now()
    print(filename," complete, it took ",end-start," seconds")
    print(df.head())
    df.to_csv(filename,sep="|",index=False)
    print("file saved")
    
fill_dataframe(df,'duplicate_questions.csv') # the original dataset

In [6]:
'''
create a spell-checked version of the entire dataframe
'''
import hunspell
from nltk.tokenize.moses import MosesDetokenizer

hobj = hunspell.HunSpell('en_US/en_US.dic', 'en_US/en_US.aff') # load the necessary dictionaries

df_spellcheck = df.copy() # new df for spellchecked version of the entire dataframe

# uses Hunspell to check each token against dictionary, replaces with most likely suggestion if token not found
def get_spellchecked(text):
    for index,token in enumerate(text):
        try:
            if token =="?":
                continue
            is_correct = hobj.spell(token)
            if not is_correct:
                correction = hobj.suggest(token)[0]
                text[index] = correction
        except:
            continue
            
    detokenizer = MosesDetokenizer()
    text = detokenizer.detokenize(text,return_str=True) # use detokenizer to get str version of text back
    
    return text

# fill the new df with spellchecked text
for i,row in df_spellcheck.iterrows():
    one_corrected = get_spellchecked(word_tokenize(row['question1']))
    df_spellcheck.loc[i,'question1'] = one_corrected
    
    two_corrected = get_spellchecked(word_tokenize(row['question2']))
    df_spellcheck.loc[i,'question2'] = two_corrected
    
fill_dataframe(df_spellcheck,'duplicate_questions_spellcheck.csv') #the spellchecked dataset


starting  duplicate_questions_spellcheck.csv
named entities complete for  duplicate_questions_spellcheck.csv , took  1:48:07.698616  seconds
                                           question1  \
0  What is the step by step guide to invest in sh...   
1  What is the story of Kohinoor e Oh-i-Noor e Di...   
2  How can I increase the speed of my INTERNET co...   
3  Why am I mentally very lonely? How can I solve...   
4  Which one dissolve in water quickly sugar e sa...   

                                           question2  is_duplicate  \
0  What is the step by step guide to invest in sh...             0   
1  What would happen if the Indian government sto...             0   
2  How can Internet speed be increased by hacking...             0   
3  Find the remainder when e math e 23^ e 24 e e ...             0   
4            Which fish would survive in salt water?             0   

   entity overlap  
0            0.00  
1            0.25  
2            0.00  
3            0.00  
4

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import operator

# gets 20 misclassified instances and prints them to file, including model's prediction
def get_error_analysis(X,y):
    file = open('error_analysis.txt','w')
    df = pd.read_csv('duplicate_questions_spellcheck.csv',delimiter='|')
    compare = list(zip(X,y))
    misclassifications = []
    for i,(X,y) in enumerate(compare):
        if X != y:
            misclassifications.append(i)
    
    indices = np.random.randint(0,high=len(misclassifications),size=20)
    
    for index in indices:
        text_one = df.loc[index,'question1']
        text_two = df.loc[index,'question2']
        file.write(text_one+"\n"+text_two+"\n")
        file.write("(model predicted "+str(compare[index][0])+")\n\n")
    file.close()

def regression_results(df,features):
    print("PRINTING ACCURACY FOR ",features,)
    filters = ['question1','question2']
    y = df['is_duplicate']
    df = df.drop(['is_duplicate'],axis=1)
    
    cols = []
    
    if features is not None:
        cols = list(df.columns)
        for feature in features:
            cols.remove(feature)
    
    filters += cols
    
    X = df[df.columns.difference(filters)] # filter out the text of the questions for analysis
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    logisticRegr = LogisticRegression()
    logisticRegr.fit(X_train, y_train)
    predictions = logisticRegr.predict(X_test)
    
    if features == None:
        get_error_analysis(predictions,y_test)
    
    score = logisticRegr.score(X_test, y_test)
    print("%.3f" % (score*100),"%")

    cols = X.columns
    stdev = np.std(X_train, 0)
    stdev_vals = [stdev[i] for i,val in enumerate(stdev)]
    stdev_vals = stdev_vals*logisticRegr.coef_
    for i,val in enumerate(stdev_vals[0]):
        stdev[i] = val

    col_stdev_dict = dict([k,v] for k,v in list(zip(cols,stdev)))
    print(sorted(col_stdev_dict.items(), key=operator.itemgetter(1),reverse=True)[:3])
    print("\n")

files = ['duplicate_questions.csv','duplicate_questions_spellcheck.csv']

for file in files:
    df = pd.read_csv(file,delimiter='|')
    
    #overall accuracy results
    regression_results(df,None)

    #entity overlap accuracy
    regression_results(df,['entity overlap'])

    #question word accuracy
    question_words = ['what/which','when','where','who/whom','why','how']
    # each word separately
    for word in question_words:
        regression_results(df,[word])
    
    #all question words together
    regression_results(df,question_words)

    #ngram accuracy by gram (cosine and jaccardian), and each measure separately
    ngrams = ['1-gram','2-gram','3-gram','4-gram','5-gram','6-gram']
    for gram in ngrams:
        regression_results(df,[gram+' cosine',gram+' jaccard'])
        regression_results(df,[gram+' cosine'])
        regression_results(df,[gram+' jaccard'])

    # all ngrams together
    jaccard_list = []
    cosine_list = []
    for gram in ngrams:
        jaccard_list.append(gram+' jaccard')
        cosine_list.append(gram+' cosine')
    regression_results(df,cosine_list)
    regression_results(df,jaccard_list)
    regression_results(df,cosine_list+jaccard_list)

PRINTING ACCURACY FOR  None
69.245 %
[('2-gram cosine', 2.8835302409187547), ('1-gram jaccard', 1.2473903254071896), ('6-gram jaccard', 0.4549311888569346)]


PRINTING ACCURACY FOR  ['entity overlap']
63.108 %
[('entity overlap', 0.2144051319281528)]


PRINTING ACCURACY FOR  ['what/which']
63.130 %
[('what/which', 0.07806939000196023)]


PRINTING ACCURACY FOR  ['when']
63.130 %
[('when', -0.05607662217233648)]


PRINTING ACCURACY FOR  ['where']
63.130 %
[('where', -0.02010545912858436)]


PRINTING ACCURACY FOR  ['who/whom']
63.130 %
[('who/whom', 0.003651326480146271)]


PRINTING ACCURACY FOR  ['why']
63.135 %
[('why', 0.07892364303259582)]


PRINTING ACCURACY FOR  ['how']
63.577 %
[('how', 0.20180725517357606)]


PRINTING ACCURACY FOR  ['what/which', 'when', 'where', 'who/whom', 'why', 'how']
63.509 %
[('how', 0.35629421406900247), ('what/which', 0.27796667705780065), ('why', 0.20166324692350385)]


PRINTING ACCURACY FOR  ['1-gram cosine', '1-gram jaccard']
64.770 %
[('1-gram cosine',

In [8]:
'''
Tensorflow implementation
Tutorial provided by:
http://adventuresinmachinelearning.com/word2vec-tutorial-tensorflow/
'''
import tensorflow as tf
import math
from collections import Counter, deque
import random

# get a list of all words in the dataframe for word2vec processing, and call build_dataset to extract n most 
# common words and assign unique integer to each word.  Returns dicts which can look up integer-word and 
# word-integer pairs.
def get_data(df,vocabulary_size):
    vocabulary = []
    
    for i,row in df.iterrows():
        tok_text = word_tokenize(row['question1']) + word_tokenize(row['question2'])
        vocabulary += tok_text
                                    
    data, count, dictionary, reverse_dictionary = build_dataset(vocabulary,
                                                                vocabulary_size)
    del vocabulary  # Hint to reduce memory.

    return data, count, dictionary, reverse_dictionary

# Extract the n most common words from each level's texts.  
# Credit to http://adventuresinmachinelearning.com/word2vec-tutorial-tensorflow/
def build_dataset(words, n_words):
    count = [['UNK', -1]]
    count.extend(Counter(words).most_common(n_words - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

data_index = 0
# generate batch data
def generate_batch(data, batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    context = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # [ skip_window input_word skip_window ]
    buffer = deque(maxlen=span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    for i in range(batch_size // num_skips):
        target = skip_window  # input word at the center of the buffer
        targets_to_avoid = [skip_window]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]  # this is the input word
            context[i * num_skips + j, 0] = buffer[target]  # these are the context words
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    # Backtrack a little bit to avoid skipping words in the end of a batch
    data_index = (data_index + len(data) - span) % len(data)
    return batch, context

vocabulary_size = 10000
data, count, dictionary, reverse_dictionary = get_data(df, vocabulary_size=vocabulary_size)

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64    # Number of negative examples to sample.

batch_size = 128
embedding_size = 128  # Dimension of the embedding vector.
skip_window = 1       # How many words to consider left and right.
num_skips = 2         # How many times to reuse an input to generate a context

import datetime as dt

graph = tf.Graph()

def run(graph, num_steps):
    with tf.Session(graph=graph) as session:
        # We must initialize all variables before we use them.
        init.run()
        print('Initialized')

        average_loss = 0
        for step in range(num_steps):
            batch_inputs, batch_context = generate_batch(data,
                batch_size, num_skips, skip_window)
            feed_dict = {train_inputs: batch_inputs, train_context: batch_context}

            # We perform one update step by evaluating the optimizer op (including it
            # in the list of returned values for session.run()
            _, loss_val = session.run([optimizer, cross_entropy], feed_dict=feed_dict)
            average_loss += loss_val

            if step % 2000 == 0:
                if step > 0:
                    average_loss /= 2000
                # The average loss is an estimate of the loss over the last 2000 batches.
                print('Average loss at step ', step, ': ', average_loss)
                average_loss = 0
    
                # Note that this is expensive (~20% slowdown if computed every 500 steps)
            if step % 10000 == 0:
                sim = similarity.eval()
                for i in range(valid_size):
                    valid_word = reverse_dictionary[valid_examples[i]]
                    top_k = 8  # number of nearest neighbors
                    nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                    log_str = 'Nearest to %s:' % valid_word
                    for k in range(top_k):
                        close_word = reverse_dictionary[nearest[k]]
                        log_str = '%s %s,' % (log_str, close_word)
                    print(log_str)
        
        final_embeddings = normalized_embeddings.eval()
            
with graph.as_default():

    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_context = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    
    # Look up embeddings for inputs.
    embeddings = tf.Variable(
        tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)
    
    # Construct the variables for the softmax
    weights = tf.Variable(
    tf.truncated_normal([embedding_size, vocabulary_size],
                          stddev=1.0 / math.sqrt(embedding_size)))
    biases = tf.Variable(tf.zeros([vocabulary_size]))
    hidden_out = tf.transpose(tf.matmul(tf.transpose(weights), tf.transpose(embed))) + biases

    # convert train_context to a one-hot format
    train_one_hot = tf.one_hot(train_context, vocabulary_size)

    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hidden_out, labels=train_one_hot))
    
    # Compute the cosine similarity between minibatch examples and all embeddings.
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keepdims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(
        normalized_embeddings, valid_dataset)
    similarity = tf.matmul(
        valid_embeddings, normalized_embeddings, transpose_b=True)

    # Construct the variables for the NCE loss
    nce_weights = tf.Variable(
        tf.truncated_normal([vocabulary_size, embedding_size],
                            stddev=1.0 / math.sqrt(embedding_size)))
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

    nce_loss = tf.reduce_mean(
        tf.nn.nce_loss(weights=nce_weights,
                       biases=nce_biases,
                       labels=train_context,
                       inputs=embed,
                       num_sampled=num_sampled,
                       num_classes=vocabulary_size))

    optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(nce_loss)

    # Add variable initializer.
    init = tf.global_variables_initializer()

num_steps = 50000
nce_start_time = dt.datetime.now()
run(graph,num_steps)
nce_end_time = dt.datetime.now()
print("NCE method took {} seconds to run 50000 iterations".format((nce_end_time-nce_start_time).total_seconds()))

/Users/andrewbeaton/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Initialized
Average loss at step  0 :  9.345418930053711
Nearest to UNK: reputation, commit, east, banned, Dual, object-oriented, remotely, flagged,
Nearest to if: pleasure, weights, induction, prevalent, logic, Power, matches, calcium,
Nearest to Who: Seals, regional, innovative, engaged, Course, Comics, Ellen, prescribe,
Nearest to think: KP, nude, progress, conclusion, kit, justice, registration, bark,
Nearest to good: yogurt, publishing, mind-blowing, circles, baseball, officially, insane, depression,
Nearest to all: voted, classified, 64, AIMS, Metal, laptops, advertise, unexpected,
Nearest to I: market, participate, everyone, doctor, Bier, holidays, expenditure, tower,
Nearest to money: Energy, dreaming, Chechen, minute, Somme, provides, Don, TATS,
Nearest to by: legacy, Brazil, passports, A